In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
data = pd.concat([train,test]).reset_index(drop=True)

In [4]:
data.head(3)

,ID,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault
0,ID_QF0ZTQJ2SF5Q,2021-11-03 04:06:31,52.58,49.52,17.4,96.0,0.0
1,ID_4GTK689CNX5S,2021-11-08 18:43:23,35.25,33.40,25.0,75.0,0.0
2,ID_DL7VVKW9U7XQ,2021-11-07 09:50:33,19.18,23.50,24.9,75.0,0.0


In [5]:
data['created_at'] = pd.to_datetime(data['Datetime'])
data['year'] = data['created_at'].dt.year
data['year'] = data['year'].astype(float)
data['month'] = data['created_at'].dt.month
data['day'] = data['created_at'].dt.day
data['weekday'] = data['created_at'].dt.weekday
data['weekofyear'] = data['created_at'].dt.weekofyear
data['hour'] = data['created_at'].dt.hour

C:\Users\Hp\AppData\Local\Temp/ipykernel_7504/4153536749.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['weekofyear'] = data['created_at'].dt.weekofyear


In [6]:
# combination between time features
data['day_hour']  = data['day'].astype(str) + '-' + data['hour'].astype(str)
data['month_day'] = data['month'].astype(str) + '-' + data['day'].astype(str)

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
def LE(df,LE_cols = []) :
    LE = LabelEncoder()
    for le_col in LE_cols :
        data[le_col] = LE.fit_transform(data[le_col])

In [9]:
# Label Encoder
LE(data,LE_cols = ['day_hour','month_day'])

In [10]:
# Feature Engineering
data = data.sort_values('Datetime').reset_index(drop=True)

In [11]:
data.head(3)

,ID,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault,created_at,year,month,day,weekday,weekofyear,hour,day_hour,month_day
0,ID_37STUCVVPZKP,2021-10-15 16:00:31,9.17,9.28,29.9,53.0,0.0,2021-10-15 16:00:31,2021.0,10,15,4,41,16,152,30
1,ID_YZZUOMF0TQIF,2021-10-15 16:00:34,9.93,11.25,29.9,53.0,0.0,2021-10-15 16:00:34,2021.0,10,15,4,41,16,152,30
2,ID_UF68QLIPG3LE,2021-10-15 16:01:01,6.53,3.15,29.9,53.0,1.0,2021-10-15 16:01:01,2021.0,10,15,4,41,16,152,30


In [12]:
def LAG(data,LagFeature,shift=1,NewFeatures=[]) :
    data[NewFeatures[0]]   = data[LagFeature]  - data[LagFeature].shift(shift)
    data[NewFeatures[1]]   = data[LagFeature].shift(shift)

In [13]:
LAG(data,LagFeature='Sensor1_PM2.5',shift=1,NewFeatures=['sensor1_diff_next','sensor1_next'])
LAG(data,LagFeature='Sensor1_PM2.5',shift=-1,NewFeatures=['sensor1_diff_before','sensor1_before'])
LAG(data,LagFeature='Sensor2_PM2.5',shift=1,NewFeatures=['sensor2_diff_next','sensor2_next'])
LAG(data,LagFeature='Sensor2_PM2.5',shift=-1,NewFeatures=['sensor2_diff_before','sensor2_before'])

In [14]:
data['SplitBy']   = data['year'].astype(int).astype(str) + '-' + data['month'].astype(str) + '-' + data['day'].astype(str)

In [15]:
data = data.sort_values('SplitBy').reset_index(drop=True)

In [16]:
data.head(3)

,ID,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault,created_at,year,month,...,month_day,sensor1_diff_next,sensor1_next,sensor1_diff_before,sensor1_before,sensor2_diff_next,sensor2_next,sensor2_diff_before,sensor2_before,SplitBy
0,ID_37STUCVVPZKP,2021-10-15 16:00:31,9.17,9.28,29.9,53.0,0.0,2021-10-15 16:00:31,2021.0,10,...,30,NaN,NaN,-0.76,9.93,NaN,NaN,-1.97,11.25,2021-10-15
1,ID_FFR983O9P6PN,2021-10-15 21:28:47,66.65,65.68,21.7,83.0,0.0,2021-10-15 21:28:47,2021.0,10,...,30,12.7,53.95,15.60,51.05,20.23,45.45,22.81,42.87,2021-10-15
2,ID_IOIOHCC348SZ,2021-10-15 21:29:48,51.05,42.87,21.7,83.0,1.0,2021-10-15 21:29:48,2021.0,10,...,30,-15.6,66.65,-19.52,70.57,-22.81,65.68,-30.41,73.28,2021-10-15


In [17]:
train = data[data['ID'].isin(train['ID'].values)].reset_index(drop=True)
train['Offset_fault'] = train['Offset_fault'].astype('int')
test = data[~data['ID'].isin(train['ID'].values)].reset_index(drop=True)

In [18]:
class CFG :
  SEED = 42
  n_splits = 5

  lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                'n_estimators': 2500,'reg_lambda' :50,'num_leaves' : 81,
                'seed': SEED,'silent':True,'early_stopping_rounds': 100,
               }
  remove_features = ['ID', 'Datetime',"created_at",'SplitBy' , 'folds', 'Offset_fault',]
  TARGET_COL = 'Offset_fault'

In [19]:
class CustomSplit :
  def __init__(self,) :
    self.n_splits = CFG.n_splits

  def Split(self,Train_) :
    kf = GroupKFold(n_splits=self.n_splits)

    Train = Train_.copy()
    Train = Train.drop_duplicates('SplitBy').reset_index(drop=True)
    
    groups = Train['SplitBy']
    Train["folds"]=-1   
    for fold, (_, val_index) in enumerate(kf.split(Train,Train['Offset_fault'],groups)):
          Train.loc[val_index, "folds"] = fold
    return Train

  def apply(self,train) :
    mapper = dict(zip(self.Split(train)['SplitBy'].tolist(),
                      self.Split(train)['folds'].tolist()))

    train['folds'] = train['SplitBy'].map(mapper)
    return train

In [20]:
from sklearn.model_selection import GroupKFold

In [22]:
split = CustomSplit() 

train = split.apply(train)

In [23]:
train.head(3)

,ID,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault,created_at,year,month,...,sensor1_diff_next,sensor1_next,sensor1_diff_before,sensor1_before,sensor2_diff_next,sensor2_next,sensor2_diff_before,sensor2_before,SplitBy,folds
0,ID_37STUCVVPZKP,2021-10-15 16:00:31,9.17,9.28,29.9,53.0,0,2021-10-15 16:00:31,2021.0,10,...,NaN,NaN,-0.76,9.93,NaN,NaN,-1.97,11.25,2021-10-15,3
1,ID_FFR983O9P6PN,2021-10-15 21:28:47,66.65,65.68,21.7,83.0,0,2021-10-15 21:28:47,2021.0,10,...,12.7,53.95,15.60,51.05,20.23,45.45,22.81,42.87,2021-10-15,3
2,ID_IOIOHCC348SZ,2021-10-15 21:29:48,51.05,42.87,21.7,83.0,1,2021-10-15 21:29:48,2021.0,10,...,-15.6,66.65,-19.52,70.57,-22.81,65.68,-30.41,73.28,2021-10-15,3


In [24]:
features_columns = [col for col in train.columns if col not in CFG.remove_features]
len(features_columns)

20

In [31]:
x = train[features_columns]
y = train["Offset_fault"]

In [33]:
(x.shape,y.shape)

((297177, 20), (297177,))

### Model

In [34]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.1,stratify= y,random_state = 42)

In [38]:
(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

((267459, 20), (267459,), (29718, 20), (29718,))

In [39]:
skf = RepeatedStratifiedKFold(n_splits=5, random_state=None)

In [54]:
test_score=[]
train_score=[]
sub_list = []

In [56]:
test = test[features_columns]

In [57]:
for train_index, test_index in skf.split(x_train,y_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x_train.iloc[train_index], x_train.iloc[test_index]
    Y_train, Y_test = y_train.iloc[train_index], y_train.iloc[test_index]
    
    
    model = LGBMClassifier() 
    #eval_dataset = lgb.Dataset(x_val, label=y_val)
    
    model.fit(X_train,Y_train,eval_set= [(X_test,Y_test)],eval_metric="accuracy")
    train_preds = model.predict(X_train)
    print("Train Accuracy score: ",accuracy_score(Y_train,train_preds))
    train_score.append(np.mean(accuracy_score(Y_train,train_preds)))
    
    test_preds = model.predict(X_test)
    print("Test Accuracy score: ", accuracy_score(Y_test,test_preds))
    test_score.append(np.mean(accuracy_score(Y_test,test_preds)))

    submission = model.predict(test)
    sub_list.append(submission) 

[1]	valid_0's binary_logloss: 0.568199
[2]	valid_0's binary_logloss: 0.495405
[3]	valid_0's binary_logloss: 0.435457
[4]	valid_0's binary_logloss: 0.385296
[5]	valid_0's binary_logloss: 0.342758
[6]	valid_0's binary_logloss: 0.306187
[7]	valid_0's binary_logloss: 0.274426
[8]	valid_0's binary_logloss: 0.246651
[9]	valid_0's binary_logloss: 0.222424
[10]	valid_0's binary_logloss: 0.20105
[11]	valid_0's binary_logloss: 0.182046
[12]	valid_0's binary_logloss: 0.165141
[13]	valid_0's binary_logloss: 0.150216
[14]	valid_0's binary_logloss: 0.136804
[15]	valid_0's binary_logloss: 0.125023
[16]	valid_0's binary_logloss: 0.114356
[17]	valid_0's binary_logloss: 0.104862
[18]	valid_0's binary_logloss: 0.0964223
[19]	valid_0's binary_logloss: 0.0888522
[20]	valid_0's binary_logloss: 0.0820075
[21]	valid_0's binary_logloss: 0.0760533
[22]	valid_0's binary_logloss: 0.0705765
[23]	valid_0's binary_logloss: 0.065745
[24]	valid_0's binary_logloss: 0.0611866
[25]	valid_0's binary_logloss: 0.0571351
[26

Train Accuracy score:  0.9974295101581085
Test Accuracy score:  0.9955507365587377
[1]	valid_0's binary_logloss: 0.568073
[2]	valid_0's binary_logloss: 0.495128
[3]	valid_0's binary_logloss: 0.435136
[4]	valid_0's binary_logloss: 0.38476
[5]	valid_0's binary_logloss: 0.341998
[6]	valid_0's binary_logloss: 0.305338
[7]	valid_0's binary_logloss: 0.273335
[8]	valid_0's binary_logloss: 0.245413
[9]	valid_0's binary_logloss: 0.220991
[10]	valid_0's binary_logloss: 0.199363
[11]	valid_0's binary_logloss: 0.180481
[12]	valid_0's binary_logloss: 0.163694
[13]	valid_0's binary_logloss: 0.148693
[14]	valid_0's binary_logloss: 0.135309
[15]	valid_0's binary_logloss: 0.123411
[16]	valid_0's binary_logloss: 0.112784
[17]	valid_0's binary_logloss: 0.103219
[18]	valid_0's binary_logloss: 0.0947079
[19]	valid_0's binary_logloss: 0.0870805
[20]	valid_0's binary_logloss: 0.0801832
[21]	valid_0's binary_logloss: 0.0740932
[22]	valid_0's binary_logloss: 0.0686531
[23]	valid_0's binary_logloss: 0.0636838
[

[100]	valid_0's binary_logloss: 0.0130916
Train Accuracy score:  0.9973453850360102
Test Accuracy score:  0.9957937635534285
[1]	valid_0's binary_logloss: 0.568118
[2]	valid_0's binary_logloss: 0.495061
[3]	valid_0's binary_logloss: 0.434967
[4]	valid_0's binary_logloss: 0.384693
[5]	valid_0's binary_logloss: 0.341987
[6]	valid_0's binary_logloss: 0.305034
[7]	valid_0's binary_logloss: 0.273174
[8]	valid_0's binary_logloss: 0.245295
[9]	valid_0's binary_logloss: 0.220958
[10]	valid_0's binary_logloss: 0.199451
[11]	valid_0's binary_logloss: 0.180512
[12]	valid_0's binary_logloss: 0.16372
[13]	valid_0's binary_logloss: 0.148934
[14]	valid_0's binary_logloss: 0.135646
[15]	valid_0's binary_logloss: 0.123727
[16]	valid_0's binary_logloss: 0.113122
[17]	valid_0's binary_logloss: 0.103623
[18]	valid_0's binary_logloss: 0.0951562
[19]	valid_0's binary_logloss: 0.0875937
[20]	valid_0's binary_logloss: 0.0806931
[21]	valid_0's binary_logloss: 0.0746118
[22]	valid_0's binary_logloss: 0.0691794


[100]	valid_0's binary_logloss: 0.0137817
Train Accuracy score:  0.9974996144265237
Test Accuracy score:  0.9955133477903237
[1]	valid_0's binary_logloss: 0.56811
[2]	valid_0's binary_logloss: 0.495218
[3]	valid_0's binary_logloss: 0.435294
[4]	valid_0's binary_logloss: 0.385112
[5]	valid_0's binary_logloss: 0.342429
[6]	valid_0's binary_logloss: 0.305902
[7]	valid_0's binary_logloss: 0.273812
[8]	valid_0's binary_logloss: 0.245869
[9]	valid_0's binary_logloss: 0.22148
[10]	valid_0's binary_logloss: 0.200056
[11]	valid_0's binary_logloss: 0.181077
[12]	valid_0's binary_logloss: 0.16433
[13]	valid_0's binary_logloss: 0.149386
[14]	valid_0's binary_logloss: 0.136106
[15]	valid_0's binary_logloss: 0.124339
[16]	valid_0's binary_logloss: 0.113705
[17]	valid_0's binary_logloss: 0.104123
[18]	valid_0's binary_logloss: 0.0955766
[19]	valid_0's binary_logloss: 0.0878997
[20]	valid_0's binary_logloss: 0.0809945
[21]	valid_0's binary_logloss: 0.0748253
[22]	valid_0's binary_logloss: 0.0692559
[2

Train Accuracy score:  0.9974388573938971
Test Accuracy score:  0.9960180961639123
[1]	valid_0's binary_logloss: 0.568104
[2]	valid_0's binary_logloss: 0.495203
[3]	valid_0's binary_logloss: 0.435177
[4]	valid_0's binary_logloss: 0.384983
[5]	valid_0's binary_logloss: 0.342216
[6]	valid_0's binary_logloss: 0.305455
[7]	valid_0's binary_logloss: 0.273483
[8]	valid_0's binary_logloss: 0.245541
[9]	valid_0's binary_logloss: 0.221258
[10]	valid_0's binary_logloss: 0.199677
[11]	valid_0's binary_logloss: 0.180807
[12]	valid_0's binary_logloss: 0.1641
[13]	valid_0's binary_logloss: 0.149108
[14]	valid_0's binary_logloss: 0.135676
[15]	valid_0's binary_logloss: 0.123764
[16]	valid_0's binary_logloss: 0.113126
[17]	valid_0's binary_logloss: 0.103588
[18]	valid_0's binary_logloss: 0.0951548
[19]	valid_0's binary_logloss: 0.0875554
[20]	valid_0's binary_logloss: 0.080863
[21]	valid_0's binary_logloss: 0.0747601
[22]	valid_0's binary_logloss: 0.069324
[23]	valid_0's binary_logloss: 0.0643503
[24]

...

Train Accuracy score:  0.9974341837760028
Test Accuracy score:  0.9957189860166006
[1]	valid_0's binary_logloss: 0.5682
[2]	valid_0's binary_logloss: 0.495369
[3]	valid_0's binary_logloss: 0.435436
[4]	valid_0's binary_logloss: 0.385127
[5]	valid_0's binary_logloss: 0.342585
[6]	valid_0's binary_logloss: 0.306021
[7]	valid_0's binary_logloss: 0.274362
[8]	valid_0's binary_logloss: 0.246477
[9]	valid_0's binary_logloss: 0.222159
[10]	valid_0's binary_logloss: 0.200614
[11]	valid_0's binary_logloss: 0.181702
[12]	valid_0's binary_logloss: 0.164866
[13]	valid_0's binary_logloss: 0.149905
[14]	valid_0's binary_logloss: 0.136701
[15]	valid_0's binary_logloss: 0.124803
[16]	valid_0's binary_logloss: 0.114143
[17]	valid_0's binary_logloss: 0.104544
[18]	valid_0's binary_logloss: 0.0960448
[19]	valid_0's binary_logloss: 0.0883948
[20]	valid_0's binary_logloss: 0.0814859
[21]	valid_0's binary_logloss: 0.0753052
[22]	valid_0's binary_logloss: 0.0699287
[23]	valid_0's binary_logloss: 0.0649501
[2

Train Accuracy score:  0.997476246337052
Test Accuracy score:  0.9953637927166679
[1]	valid_0's binary_logloss: 0.568137
[2]	valid_0's binary_logloss: 0.495162
[3]	valid_0's binary_logloss: 0.435229
[4]	valid_0's binary_logloss: 0.384867
[5]	valid_0's binary_logloss: 0.34227
[6]	valid_0's binary_logloss: 0.305225
[7]	valid_0's binary_logloss: 0.273222
[8]	valid_0's binary_logloss: 0.245399
[9]	valid_0's binary_logloss: 0.220921
[10]	valid_0's binary_logloss: 0.199456
[11]	valid_0's binary_logloss: 0.180536
[12]	valid_0's binary_logloss: 0.163786
[13]	valid_0's binary_logloss: 0.148889
[14]	valid_0's binary_logloss: 0.135614
[15]	valid_0's binary_logloss: 0.123768
[16]	valid_0's binary_logloss: 0.113124
[17]	valid_0's binary_logloss: 0.103618
[18]	valid_0's binary_logloss: 0.0951612
[19]	valid_0's binary_logloss: 0.0875897
[20]	valid_0's binary_logloss: 0.0807274
[21]	valid_0's binary_logloss: 0.0746916
[22]	valid_0's binary_logloss: 0.0690872
[23]	valid_0's binary_logloss: 0.0640986
[2

[99]	valid_0's binary_logloss: 0.0131624
[100]	valid_0's binary_logloss: 0.0131336
Train Accuracy score:  0.997490267190735
Test Accuracy score:  0.9957002916323936
[1]	valid_0's binary_logloss: 0.568249
[2]	valid_0's binary_logloss: 0.495266
[3]	valid_0's binary_logloss: 0.435287
[4]	valid_0's binary_logloss: 0.385073
[5]	valid_0's binary_logloss: 0.342313
[6]	valid_0's binary_logloss: 0.305245
[7]	valid_0's binary_logloss: 0.273483
[8]	valid_0's binary_logloss: 0.245581
[9]	valid_0's binary_logloss: 0.221236
[10]	valid_0's binary_logloss: 0.199808
[11]	valid_0's binary_logloss: 0.180958
[12]	valid_0's binary_logloss: 0.164007
[13]	valid_0's binary_logloss: 0.149028
[14]	valid_0's binary_logloss: 0.135666
[15]	valid_0's binary_logloss: 0.123775
[16]	valid_0's binary_logloss: 0.113168
[17]	valid_0's binary_logloss: 0.103672
[18]	valid_0's binary_logloss: 0.0951919
[19]	valid_0's binary_logloss: 0.087695
[20]	valid_0's binary_logloss: 0.0809028
[21]	valid_0's binary_logloss: 0.0747053
[

[100]	valid_0's binary_logloss: 0.0128413
Train Accuracy score:  0.9974108156865311
Test Accuracy score:  0.9957189860166006
[1]	valid_0's binary_logloss: 0.568088
[2]	valid_0's binary_logloss: 0.49502
[3]	valid_0's binary_logloss: 0.434976
[4]	valid_0's binary_logloss: 0.384474
[5]	valid_0's binary_logloss: 0.341783
[6]	valid_0's binary_logloss: 0.305205
[7]	valid_0's binary_logloss: 0.273616
[8]	valid_0's binary_logloss: 0.245606
[9]	valid_0's binary_logloss: 0.221274
[10]	valid_0's binary_logloss: 0.199723
[11]	valid_0's binary_logloss: 0.180757
[12]	valid_0's binary_logloss: 0.163946
[13]	valid_0's binary_logloss: 0.149169
[14]	valid_0's binary_logloss: 0.13587
[15]	valid_0's binary_logloss: 0.123998
[16]	valid_0's binary_logloss: 0.113373
[17]	valid_0's binary_logloss: 0.103884
[18]	valid_0's binary_logloss: 0.095431
[19]	valid_0's binary_logloss: 0.087875
[20]	valid_0's binary_logloss: 0.0811709
[21]	valid_0's binary_logloss: 0.0749471
[22]	valid_0's binary_logloss: 0.0694224
[23

Train Accuracy score:  0.9974482165557467
Test Accuracy score:  0.9957562954515713


In [58]:
print("Training Score: ",np.mean(train_score))
print("Test Score: ",np.mean(test_score))

Training Score:  0.9974068922934345
Test Score:  0.995701397491665


In [59]:
predictions = pd.DataFrame(sub_list)

In [61]:
predictions.head(3)

,0,1,2,3,4,5,6,7,8,9,...,127351,127352,127353,127354,127355,127356,127357,127358,127359,127360
0,0,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,1,0,0,0
1,0,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,1,0,0,0
2,0,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,1,0,0,0


In [ ]:
#predictions = np.where(np.mean(predictions,axis=0)> 0.45, 1, 0)

In [62]:
predictions1 = predictions.T

In [63]:
predictions1

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127357,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
127358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
